In [4]:
import numpy as np
import cv2
import csv
import os.path
import time
from os import listdir

RootPath = 'E:\\MAGL_VIDEO_CHRONIC\\Annotated0213_0223'
subdirectories = os.listdir(RootPath)
   
folderNum = len(subdirectories)

for mm in xrange(folderNum):
#for mm in range(5,23,16):
    videoNameRoot = RootPath+'\\'+subdirectories[mm]
    videoNames = listdir(videoNameRoot)
    videoName = videoNameRoot+'\\' + videoNames[0]
    print videoName


    time_start = time.time()

    #-----------------------------------------------------
    #
    # compute background
    #
    #-----------------------------------------------------

    (dirName,videoFileName) = os.path.split(videoName)
    (videoBaseName, videoExtension) = os.path.splitext(videoFileName)

    csvOut   = dirName+"\\"+videoBaseName+"_out.csv"
    fout = open(csvOut, 'wb')
    csvWriter = csv.writer(fout)
    csvWriter.writerow(['videoBaseName','ifr','cnt_area','Cx','Cy','cnt_solidity','Eccentricity','Angle','Intensity_change'])


    cap = cv2.VideoCapture(videoName)
    #fps = cap.get(cv2.CAP_PROP_FPS)
    #length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)
    fps=15
    length = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))


    ret,frame_it0 = cap.read()  
    frame_it = cv2.cvtColor(frame_it0, cv2.COLOR_BGR2GRAY)
    width = np.size(frame_it, 1) 
    height = np.size(frame_it, 0)
    print width,height

    # Specify output video
    #fourcc = cv2.cv.CV_FOURCC('M','P','4','V')
    #fourcc = cv2.cv.CV_FOURCC('I','4','2','0')
    #fourcc = cv2.cv.CV_FOURCC('M','J','P','G')
    #fourcc = cv2.cv.CV_FOURCC(*'XVID')
    #fourcc = cv2.cv.CV_FOURCC('A','V','C','1')
    #fourcc = cv2.cv.CV_FOURCC('H','2','6','4')

    fourcc = cv2.cv.CV_FOURCC(*'XVID')

    videoOut = dirName+"\\"+videoBaseName+"_out.avi"
    videoOutWriter = cv2.VideoWriter(videoOut,fourcc, 15, (width,height))

    print videoOut, fps, width,height,length


    #sys.exit(0)

    sampleFrameNums = np.linspace(1,900,10)# First 60*15 is the background
    sampleLen = len(sampleFrameNums)
    videoStack = np.zeros([height, width, sampleLen],dtype = 'uint8')
    mm = 0

    for jj in range(sampleLen): 
        #cap.set(cv2.CAP_PROP_POS_FRAMES,sampleFrameNums[jj])
        cap.set(cv2.cv.CV_CAP_PROP_POS_FRAMES,sampleFrameNums[jj])
        #cap.set(cv2.cv.CV_CAP_PROP_POS_FRAMES,1)
        ret,frame_it0 = cap.read()
        #print ret
        if ret:
            frame_it = cv2.cvtColor(frame_it0, cv2.COLOR_BGR2GRAY)
            cv2.imshow('sample frame',frame_it)
            if 0xff & cv2.waitKey(5) == 27:
                break
            videoStack[:,:,mm] = frame_it
            mm+=1

    cap.release()  

    #sys.exit(0)


    lastSlice=mm-1
    videoStack = videoStack[:,:,0:lastSlice]
    #frame_ref0 = np.amin(videoStack,axis = 2)
    #frame_ref0 = np.median(videoStack,axis = 2)
    frame_ref0 = np.percentile(videoStack,20,axis=2)
    #frame_ref0 = np.amax(videoStack,axis = 2)
    frame_base = np.uint8(frame_ref0)

    backgroundImageName =  dirName+"\\"+videoBaseName+"_out_background.tif"
    cv2.imwrite(backgroundImageName,frame_base)
    frame_base = cv2.GaussianBlur(frame_base,(5,5),1)
    #cv2.waitKey(0)


    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(20,20))

    cap = cv2.VideoCapture(videoName)
    cap.set(cv2.cv.CV_CAP_PROP_POS_FRAMES,900)
    
    ###ret,frame_prev0 = cap.read()
    
    for ii in range(0,length-901): # First 60*15 is the background
    #for ii in range(0,600):


        ret,frame0 = cap.read()
        #print ret
        #frame0.shape

        if ii==0:
            frame_prev0=frame0
        
        frame = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
        frame = cv2.GaussianBlur(frame,(5,5),1)
        
        frame_prev = cv2.cvtColor(frame_prev0, cv2.COLOR_BGR2GRAY)
        frame_prev = cv2.GaussianBlur(frame_prev,(5,5),1)        

        intensity_change = np.subtract(frame_prev.astype(np.int16),frame.astype(np.int16))
        intensity_change = np.mean(abs(intensity_change))
        
        diff_16 = np.subtract(frame.astype(np.int16),frame_base.astype(np.int16))
        diff_16 = np.piecewise(diff_16,[diff_16<0,diff_16>=0],[lambda diff_16:0, lambda diff_16:diff_16])
        diff = diff_16.astype(np.uint8)


        ret,fgmask0 = cv2.threshold(diff,40,255,cv2.THRESH_BINARY)
        
        ### Added 03/06/2017 to filter out false positive in the right for animal 41 and 53 
        ####fgmask0[:,580:639] = 0
        
        #ret,fgmask0 = cv2.threshold(diff,50,255,cv2.THRESH_BINARY)
        fgmask2 = cv2.morphologyEx(fgmask0,cv2.MORPH_OPEN,kernel2)
        #fgmask2 = cv2.morphologyEx(fgmask1,cv2.MORPH_OPEN,kernel)


        ###cv2.imshow('substraction0',diff)
        ###k = cv2.waitKey(10) & 0xff
        ###if k == 27:
        ###    break

        #find contours
        #_,contours,_ = cv2.findContours(fgmask2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        contours,_ = cv2.findContours(fgmask2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) > 0:

            areas = [] #list to hold all areas
            for contour in contours:
                ar = cv2.contourArea(contour)
                areas.append(ar)

            max_area = max(areas)
            max_area_index = areas.index(max_area) #index of the list element with largest area

            cnt = contours[max_area_index] #largest area contour

            if len(cnt) > 5:

                cnt_area = cv2.contourArea(cnt)
                cnt_hull_area = cv2.contourArea(cv2.convexHull(cnt))
                cnt_solidity = float(cnt_area)/cnt_hull_area
                #m = cv2.moments(cnt)
                #Centroid = ( m['m10']/m['m00'],m['m01']/m['m00'] )

                # fit ellipse
                (Cx,Cy),(Minor,Major),Angle = cv2.fitEllipse(cnt)
                Eccentricity = np.sqrt(1-(Minor/Major)**2)

               # print ii,length, cnt_area, Cx, Cy, cnt_solidity, Eccentricity, Angle,intensity change
                csvWriter.writerow((videoBaseName, ii, cnt_area, Cx, Cy, cnt_solidity, Eccentricity, Angle,intensity_change))




                iCenter = (int(Cx), int(Cy))
                AngleMajor = (Angle-90)/180*np.pi
                AngleMinor = (Angle-180)/180*np.pi
                drMajor = ( int(Major/2*np.cos(AngleMajor)), int(Major/2*np.sin(AngleMajor)))
                drMinor = ( int(Minor/2*np.cos(AngleMinor)), int(Minor/2*np.sin(AngleMinor)))

                ellipse = cv2.fitEllipse(cnt)
                cv2.drawContours(frame0, [cnt], 0, (0, 255, 0), 3, maxLevel = 0)
                cv2.ellipse(frame0,ellipse,(0,0,255),2)
                cv2.circle(frame0,iCenter,5,[0,0,255],-1)
                cv2.putText(frame0,str(int(Angle)), (iCenter[0]+5,iCenter[1]+5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, 255)
                cv2.line(frame0, iCenter, (iCenter[0]+drMajor[0], iCenter[1]+drMajor[1]), (0, 0, 255))
                cv2.line(frame0, iCenter, (iCenter[0]+drMinor[0], iCenter[1]+drMinor[1]), (255, 0, 0))
                videoOutWriter.write(frame0)



        frame_prev0 = frame0
        #cv2.imshow('frame '+str(ii), frame0)
        ###cv2.imshow('',frame0)
        #videoOutWriter.write(frame0)

        ###k = cv2.waitKey(1) & 0xff
        ###if k == 27:
        ###    break

    time_end = time.time()
    print time_end - time_start


    cap.release()
    videoOutWriter.release()
    cv2.destroyAllWindows()

E:\MAGL_VIDEO_CHRONIC\Annotated0213_0223\P41-Stimulated\P41-Stimulated_Box4_2017-02-13_22291.avi
640 360
E:\MAGL_VIDEO_CHRONIC\Annotated0213_0223\P41-Stimulated\P41-Stimulated_Box4_2017-02-13_22291_out.avi 15 640 360 37904
480.778000116
E:\MAGL_VIDEO_CHRONIC\Annotated0213_0223\P53-Stimulated\P53-Stimulated_Box4_2017-02-23_77611.avi
640 360
E:\MAGL_VIDEO_CHRONIC\Annotated0213_0223\P53-Stimulated\P53-Stimulated_Box4_2017-02-23_77611_out.avi 15 640 360 36223
461.638000011
